In [14]:
# libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
np.random.seed(32)

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.utils.np_utils import to_categorical
from keras.layers.embeddings import Embedding

## Plotly

# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)

# Others
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
import os
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE


%matplotlib inline

C:\Users\Tapas\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Tapas\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [1]:
import pandas as pd
import glob as gb
import re as re

In [2]:
files = gb.glob("Combined/data/*",recursive = False)
label = list()
content =  list()

In [3]:
for i in files:
    label.append((i[i.find('_')+1:]))
    with open(i,'r') as x:
        content.append(x.read())

In [4]:
print(content[0:3])

["\n\nIn article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:\n\n|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:\n\n|> : \tI have been active in defending Saturn lately on the net and would\n\n|> : like to state my full opinion on the subject, rather than just reply to others'\n\n|> : points.\n\n|> : \t\n\n|> : \tThe biggest problem some people seem to be having is that Saturn\n\n|> : Dealers make ~$2K on a car.  I think most will agree with me that the car is\n\n|> : comparably priced with its competitors, that is, they aren't overpriced \n\n|> : compared to most cars in their class.  I don't understand the point of \n\n|> : arguing over whether the dealer makes the $2K or not?  \n\n|> \n\n|> I have never understood what the big deal over dealer profits is either.\n\n|> The only thing that I can figure out is that people believe that if\n\n|> they minimize the dealer profit they will minimize their total out-of-pocket\n\n|> expenses for the car. While

In [17]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", "  ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text


In [22]:
clean_text(content[2])

'thank respond post the problem truck headlight low beam problem loos wire connect fuse minor suggest thank again'

In [23]:
c1 = [clean_text(l) for l in content]

In [24]:
df = pd.DataFrame({'target':label,'document':c1},columns=['target','document'])

In [26]:
df.iloc[:,]

,target,document
0,2,in articl <c4vir5 l3r shuksan ds boe com fredd...
1,2,in articl <1993apr5 135153 11132 wdl loral com...
2,2,thank respond post the problem truck headlight...
3,2,the subject say all 1984 chev s10 pickup left ...
4,2,nntp - post - host : blacker caltech edu wolfs...
5,2,nntp - post - host : hprnd rose hp com x - new...
6,2,i am look replac radio tape player 1984 toyota...
7,2,i think get infin g20 in consum report rank hi...
8,2,in articl <1993apr5 175719 7892 telxon mis tel...
9,2,in articl <1993apr5 212645 15988 nntpd lkg dec...


In [1]:
train = df.sample(frac=0.7)
test = df.drop(train.index)
print(train.shape)
print(test.shape)
print(df.shape)
train['document']

NameError: name 'df' is not defined

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
train_X = list(train['document'])
train_Y = list(train['target'])

In [30]:
vectorizer = TfidfVectorizer(input = train_X,lowercase=True,min_df=100,ngram_range=(1,4),stop_words='english')

In [31]:
vectorizer.fit(train_X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=['distribut : world messag - id : <1r36r7 i59 horus ap mchp sni de refer : <1qkovl k fido asd sgi com <1qu485 58o horus ap mchp sni de <1r2gi8 6e1 fido asd sgi com nntp - post - host : d012s658 ap mchp sni de in articl <1r2gi8 6e1 fido asd sgi com livesey solntz wpd sgi com jon livesey write :...mileag like though where found reson abl stand gaff emd ham almanac bc ca robert smit ladysmith bc'],
        lowercase=True, max_df=1.0, max_features=None, min_df=100,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [32]:
import numpy as np

In [33]:
np.asarray(train_Y).astype('int32').min()

1

In [34]:
train_mat = vectorizer.transform(train_X)

In [35]:
test_X = list(test['document'])
test_Y = list(test['target'])
test_mat = vectorizer.transform(test_X)

In [36]:
test_mat.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.36384905, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [37]:
from sklearn.cluster import KMeans

In [38]:
kmeans = KMeans(init='k-means++' ,n_clusters=6)

In [39]:
kmeans.fit(train_mat.todense(),y=train_Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [40]:
np.min(np.asarray(train_Y).astype('int32'))

1

In [41]:
from sklearn.metrics import classification

In [42]:
train_Y = [x-1 for x in np.asarray(train_Y).astype('int32')]

In [43]:
np.asarray(train_Y).astype('int32').min()

0

In [44]:
#classification.accuracy_score(y_pred=list(kmeans.labels_.astype('int32')),y_true=train_Y)
classification.recall_score(y_pred=list(kmeans.labels_.astype('int32')),y_true=train_Y,average='weighted')

0.11410837908808785

In [45]:
## min_df =10 , got 20.2 accuracy so increasing the min_df so that stop words are minimized
vectorizer1 = TfidfVectorizer(input = train_X,lowercase=True,min_df=5,ngram_range=(1,2),stop_words='english')

In [46]:
vectorizer1.fit(train_X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=['distribut : world messag - id : <1r36r7 i59 horus ap mchp sni de refer : <1qkovl k fido asd sgi com <1qu485 58o horus ap mchp sni de <1r2gi8 6e1 fido asd sgi com nntp - post - host : d012s658 ap mchp sni de in articl <1r2gi8 6e1 fido asd sgi com livesey solntz wpd sgi com jon livesey write :...mileag like though where found reson abl stand gaff emd ham almanac bc ca robert smit ladysmith bc'],
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [47]:
train_mat1 = vectorizer1.transform(train_X)

In [48]:
kmeans = KMeans(init='k-means++' ,n_clusters=6)

In [49]:
kmeans.fit(train_mat.todense())

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [50]:
pred_Y = kmeans.predict(train_mat.todense())

In [51]:
classification.accuracy_score(y_pred=pred_Y,y_true= train_Y)

0.08092623537837193

In [52]:
## min_df =10 , got 8.7 accuracy so increasing the min_df resulted in less accuracy so trying decreasing the min_df to 2
vectorizer2 = TfidfVectorizer(input = train_X,lowercase=True,min_df=5,ngram_range=(1,2),stop_words='english')

In [53]:
vectorizer2.fit(train_X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=['distribut : world messag - id : <1r36r7 i59 horus ap mchp sni de refer : <1qkovl k fido asd sgi com <1qu485 58o horus ap mchp sni de <1r2gi8 6e1 fido asd sgi com nntp - post - host : d012s658 ap mchp sni de in articl <1r2gi8 6e1 fido asd sgi com livesey solntz wpd sgi com jon livesey write :...mileag like though where found reson abl stand gaff emd ham almanac bc ca robert smit ladysmith bc'],
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [54]:
train_mat2 = vectorizer2.transform(train_X)

In [55]:
kmeans.fit(X=train_mat2.todense(),y=train_Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [56]:
pred_Y = kmeans.predict(train_mat2.todense())

In [57]:
pred_Y

array([5, 3, 4, ..., 1, 5, 4])

In [58]:
classification.accuracy_score(y_pred=pred_Y,y_true=train_Y)

0.08570064454523753

In [59]:
# Applying naive Bayes 

from sklearn.naive_bayes import MultinomialNB

In [60]:
mb = MultinomialNB()

In [61]:
mb.fit(X=train_mat.todense(),y=train_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [62]:
pred_Y = mb.predict(X=train_mat.todense())

In [63]:
classification.accuracy_score(y_pred=pred_Y,y_true=train_Y)

0.8770589639532108

In [64]:
mb.fit(X=train_mat2.todense(),y=train_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
pred_Y = mb.predict(X=train_mat2.todense())

In [66]:
classification.accuracy_score(y_pred=pred_Y,y_true=train_Y)

0.9785151587491048

In [67]:
mb.fit(X=train_mat1.todense(),y=train_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [68]:
pred_Y = mb.predict(X=train_mat1.todense())

In [69]:
classification.accuracy_score(y_pred=pred_Y,y_true=train_Y)

0.9785151587491048

In [70]:
test_mat1 = vectorizer1.transform(test_X)

In [71]:
pred_test_Y = mb.predict(X=test_mat1.todense())

In [72]:
pred_test_Y
test_Y = [x-1 for x in np.asarray(test_Y).astype('int32')]

In [73]:
classification.accuracy_score(y_pred=pred_test_Y,y_true=test_Y)

0.947075208913649

In [74]:
classification.recall_score(y_pred=pred_test_Y,y_true=test_Y,average='weighted')

0.947075208913649

In [75]:
import nltk

In [78]:
def read_corpus(content):
    text = list()
    [text.append(nltk.pos_tag(nltk.word_tokenize(clean_text(str.lower())))) for str in content]
    return(text)

In [79]:
text = read_corpus(content)

In [ ]:
len(text)

In [498]:
text[]

[('sure', 'JJ'),
 ('sounds', 'VBZ'),
 ('like', 'IN'),
 ('they', 'PRP'),
 ('got', 'VBD'),
 ('a', 'DT'),
 ('ringer', 'NN'),
 ('the', 'DT'),
 ('is', 'VBZ'),
 ('i', 'JJ'),
 ('drove', 'NN'),
 ('was', 'VBD'),
 ('definitely', 'RB'),
 ('faster', 'RBR'),
 ('than', 'IN'),
 ('that', 'DT'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('want', 'VBP'),
 ('to', 'TO'),
 ('quote', 'VB'),
 ('numbers', 'NNS'),
 ('my', 'PRP$'),
 ('aw', 'NN'),
 ('autofile', 'NN'),
 ('shows', 'NNS'),
 ('in', 'IN'),
 ('mile', 'NN'),
 ('in', 'IN'),
 ('it', 'PRP'),
 ('quotes', 'VBZ'),
 ('car', 'NN'),
 ('and', 'CC'),
 ('drivers', 'NNS'),
 ('figures', 'NNS'),
 ('of', 'IN'),
 ('and', 'CC'),
 ('oh', 'UH'),
 ('btw', 'IN'),
 ('these', 'DT'),
 ('numbers', 'NNS'),
 ('are', 'VBP'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('i', 'NN'),
 ('car', 'NN'),
 ('and', 'CC'),
 ('driver', 'NN'),
 ('rated', 'VBN'),
 ('the', 'DT'),
 ('is', 'VBZ'),
 ('at', 'IN'),
 ('at', 'IN'),
 ('after', 'IN'),
 ('k', 'NN'),
 ('miles', 'NNS'),
 ('last', 'JJ'),
 ('time', 'NN'),
 ('i',

In [510]:
adj1 = list()
for w in text:
    for j in w:
        if (j[1] == 'JJ'):
            print(j[0])
            adj1.append(j[0])

active
net
full
big
only
total
true
true
comparable
sure
other
saturn
different
important
much
i
same
important
same
isnt
other
gary
annual
sure
i
variable
dont
drive
low
loose
turn
mechanical
old
average
sweden
average
ive
other
high
mechanical
own
good
sum
bad
hard
first
only
hp
com
[
shah
robert
same
large
small
shah
ralph
toyota
tercel
fine
ive
old
new
minimal
new
similar
easy
other
relevant
email
tom
tom
corporate
east
fax
infiniti
high
many
compact
second
special
update
dual
rid
old
good
infiniti
hoffman
other
galant
new
following
check
high
monthly
slow
hot
cool
great
other
other
important
gs
last
integra
wrong
right
small
beretta
mile
high
top
due
exact
low
high
torque
tested
standard
available
reliable
quad
reliable
reliable
reliable
safe
integra
general
perfect
everyday
youll
i
bad
dont
nice
same
other
gm
nice
respectable
reasonable
reliable
other
i
grand
quad
similair
half
lucas
general
san
diego
busy
ultimate
peculiar
rear
f
o
nice
big
same
local
i
rare
i
vaxvms
last
good
i

ive
moviesreal
nice
good
interstate
rare
overpopulated
arbitrary
early
american
early
early
i
i
i
other
other
dan
oswald
i
harebraned
robert
spelled
cant
quite
plastic
illinoisurbana
champaign
human
moral
selick
ive
possible
fancy
steve
able
ase
certified
fancy
manual
alignment
accurate
due
natural
ownermechanic
acceptable
simple
imaginative
pigeon
able
poor
i
magic
whole
other
similar
realignment
good
specs
difficult
upper
hard
same
last
i
original
electronic
digital
large
digital
funny
digital
short
equivalent
i
id
stupid
wdigital
digital
figure
usenet
keyless
great
great
first
i
bad
key
_
former
infamous
ct
illegal
illegal
illegal
sure
col
mechanical
i
reversible
i
locomotive
northbound
trestle
full
chirps
strong
close
enough
i
pretty
short
regular
i
own
good
stupid
sorry
own
h
frustrated
inevitable
due
tough
fourth
considerable
aviad
electric
suitable
tight
safe
leak
nj
i
plus
like
ac
third
third
ac
handy
thx
unhappy
noisy
doesnt
much
i
cant
intergraph
youre
high
pretty
middle
obvi

inadequate
mechanical
i
personal
free
single
regular
only
left
busy
plymouth
new
same
good
clear
worth
deep
lustrous
clear
other
clear
fine
several
same
i
several
added
black
red
clean
dirty
fine
visible
light
particular
peeved
previous
i
compact
full
i
serious
ahole
specific
danny
lucc
clutchless
current
toyota
ive
ive
ive
fine
tranny
old
worn
only
ive
same
i
agian
ive
high
right
right
second
gear
gross
strong
religious
such
same
underrated
eats
careful
wiper
constant
last
new
new
national
unique
new
expensive
overrated
obert
vid
enough
driven
long
e
ultimate
door
similar
civil
comfortable
late
usa
pleasant
civil
regular
harsh
front
tough
same
basic
gt
cocaine
cocaine
m
dan
audible
professional
amateur
i
heavy
false
due
cruddy
real
suburbs
other
alarmed
good
interested
april
st
alice
psuvm
male
female
female
other
interested
several
unsolicited
astronomical
good
extraordinary
jonathan
hela
edu
sentra
nx
nx
fair
only
much
same
identical
last
nxs
little
i
slight
appreciated
possible
maj

ive
mazda
reminiscent
nimble
near
id
sure
front
early
upper
shelby
interested
feedback
mustang
early
simple
upper
single
forwardbackward
simple
strut
intended
only
ken
guy
much
brian
larose
i
aware
own
good
stop
south
national
similar
worldwide
numerous
third
automatic
due
different
limited
m
medical
suite
ann
noon
possible
cute
wwii
great
i
interested
american
ford
nevada
grand
las
south
lake
cal
san
eg
private
interested
scale
amc
particular
amx
interested
interested
australian
high
interested
interested
american
ford
nevada
grand
las
south
lake
cal
san
eg
private
interested
scale
amc
particular
amx
interested
interested
australian
high
interested
eric
brian
larose
i
aware
own
good
stop
south
national
similar
worldwide
numerous
third
other
southern
last
several
many
sick
clesun
central
respectable
easy
new
expensive
false
expensive
new
real
cheap
driven
dont
apparent
many
used
jeep
irresistable
seductive
fatal
popular
previous
new
popular
popular
new
high
simple
terrible
i
unable
per

young
long
posessed
much
much
early
legs
enough
traditional
hard
easy
limited
other
fine
electrical
many
due
short
finesse
suitable
mechanical
experimental
joystick
useless
automotive
separate
i
separate
separate
lockable
same
callison
oklahoma
i
pica
tobias
basic
human
big
murderous
sick
old
cheap
little
overheated
middle
human
colored
random
ceaseless
overall
random
individual
significant
good
tamu
vaxvms
srinagesh
i
i
same
deep
i
apprecisted
mother
major
past
good
little
right
little
light
little
good
west
texas
mot
usenet
orca
alaska
arctic
anniversary
physical
welcome
welcome
arctic
o
early
alaska
tenth
anniversary
arctic
write
electronic
stablereliable
nonvolatile
independent
outside
last
say
good
mkt
current
dead
side
legal
used
replaced
k
actual
easy
new
many
correct
incorrect
correct
differential
i
unhappy
noisy
doesnt
much
i
cant
pickups
identical
other
same
probable
ac
sure
factory
installedhondas
many
cant
i
true
st
ac
good
screwdriver
usc
edu
bad
mechanic
double
sure
likel

...

next
saudi
arab
iran
good
territorial
united
continued
deep
extreme
unjustified
iranian
proclaimed
abu
small
uae
uae
whole
virtual
former
general
ready
snsc
ready
eightyear
iraq
eightyear
iraniraq
territorial
national
full
united
uae
gulf
strong
iranian
uae
kuwait
oman
iranian
friday
official
iranian
uae
irresponsible
able
iranian
english
foreign
aware
certain
uae
unfounded
abu
uae
abu
childish
abu
other
abu
af
tehran
iran
revive
tehran
radical
iranian
iraq
senior
last
united
uae
gulf
clear
kuwait
oman
uae
historical
own
iranian
foreign
serious
effective
late
independent
iranian
islamic
conservative
arab
prowestern
much
southern
persian
united
last
former
logical
saudibacked
iraqi
current
neutral
shiite
southern
long
insignificant
political
jewish
same
welcome
fellow
arab
dependent
political
same
political
fair
old
boy
old
sexist
i
purpose
effective
single
few
extremist
ineffect
nice
peaceful
welcome
ficitious
sure
old
boy
i
i
adam
big
sexual
wouldnt
edu
ed
numerous
palestine
original


wrong
only
do
powerful
moral
other
own
other
european
moral
military
rare
impassioned
many
foregin
acrimonious
religious
such
elabdellaoui
such
such
last
i
true
muslim
jewish
strong
true
local
whole
other
past
wrong
perspective
sreod
cs
michael
fresh
ethnic
intense
mutual
mutual
fresh
acceptable
balkan
bosnian
acceptable
i
acceptable
other
other
imperialist
incontrovertible
bare
public
many
found
many
predictable
complacent
other
internal
sad
sean
engelson
new
nick
loncaric
such
easy
strong
common
yugoslavia
serbian
same
serbia
nick
soft
xuseragent
subject
nick
nick
i
major
acceptable
bh
utterly
ridiculous
major
green
mladic
international
accurate
practical
impossible
nazis
extreme
genocidal
i
systematic
bosnian
muslim
many
feasible
muslims
alive
mr
major
acceptable
sure
nick
blue
dead
blue
responsible
bosnian
free
american
military
domestic
domestic
primary
international
un
ridiculous
serbs
clean
pretty
blue
dont
basic
un
valuable
bosnian
other
forced
wrong
last
decent
sick
inititiati

macedonian
ethnic
macedonian
alleged
greek
greek
criminal
athenian
greek
weekly
basic
human
ethnic
macedonian
upcoming
greek
general
greek
macedonian
territorial
main
false
unrest
public
international
greek
macedonian
ethnic
greek
macedonian
macedonian
independent
serbia
proper
long
macedonian
human
few
obscure
greek
amnesty
international
greek
florina
northern
greece
ethnic
desperate
internal
northern
last
united
bad
ethnic
western
presidential
orce
nikolov
skopje
acc
soviet
single
turkish
svr
armenian
american
east
whole
turkish
moslem
armenia
armenian
other
turkish
leonard
leonard
armenian
muslim
armenian
straw
impressed
armenian
s
papazian
baikar
own
prominent
revolutionary
successful
armenian
defenseless
innocent
turkish
armenian
turkish
armenian
caucasian
such
famous
antranik
armenian
valuable
russian
entire
little
untenable
whole
russian
hot
cold
mute
mournful
quiet
scattered
dead
appressian
entire
muslim
muslim
muslim
erzurum
muslim
diyarbakir
muslim
muslim
sivas
entire
muslim


irrelevent
other
newyork
daily
open
true
other
defence
shia
only
common
human
green
necessary
same
official
conservative
human
bezs
outside
human
saudi
main
first
official
extremist
hadith
tbph
jewish
i
own
selfhating
absolute
particular
selective
historical
complete
context
palestinian
unmoved
wrong
other
i
i
israeli
occupied
much
i
i
plestinian
israel
israeli
other
emotional
moral
reasoned
political
other
tim
uci
tmuj
umd
jewish
i
own
selfhating
absolute
particular
selective
historical
complete
context
mixed
israel
absolute
human
palestinian
jewish
suppose
jewish
palestinian
i
unmoved
wrong
other
i
i
israeli
occupied
much
i
i
plestinian
israel
israeli
other
emotional
moral
reasoned
political
other
other
guilty
other
other
other
other
other
future
etc
nick
exyugoslavia
christian
outdone
nick
davidian
white
black
alaskan
much
x
whole
difficult
staunch
serbian
ethnic
difficult
unable
civil
serb
political
negotiating
multilateral
right
outside
simple
classical
ethnic
civil
self
subject
f

optimum
russian
azerbaijani
russian
armenian
armenian
involved
russian
same
armenian
call
legal
russian
dead
simple
such
touching
russian
antiazeri
antiarmenian
accused
russian
former
mercy
armenian
innocent
new
armed
armenia
military
kafan
kedabek
same
armed
armenia
kedabek
unocal
azerbaijan
unocal
american
present
american
appropriate
personal
black
national
russian
official
national
same
present
local
financial
former
great
many
hard
much
parliamentary
last
july
presidential
new
parliamentary
new
new
supreme
soviet
republic
parliamentary
legal
fazil
shahin
national
parliamentary
present
alternative
parliamentary
isa
azerirussian
criminal
republic
isa
russian
soviet
russian
lieutenant
v
tukish
military
supreme
azerbaijan
russian
soviet
criminal
objective
such
main
russian
russian
legal
international
trilateral
russian
trilateral
same
such
russian
trilateral
initiative
russian
complete
bosnian
partial
transcaucasia
azerbaijan
complete
western
bpstatoil
caspian
complete
chirag
total
to

In [512]:
len(set(adj1))

15529

In [ ]:
TfidfVectorizer(vocabulary=)

In [80]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [81]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [82]:
train.head()

,target,document
1758,1,distribut : world messag - id : <1r36r7 i59 ho...
5931,6,in articl <30945 galaxi ucr edu raffi watnxt08...
346,2,want summer without rain wrong place must not ...
1063,1,in articl <1993apr5 024626 19942 ultb isc rit ...
2446,3,mjm hi wonder anyon would abl help twwo relat ...


In [83]:
test.head()

,target,document
0,2,in articl <c4vir5 l3r shuksan ds boe com fredd...
3,2,the subject say all 1984 chev s10 pickup left ...
4,2,nntp - post - host : blacker caltech edu wolfs...
9,2,in articl <1993apr5 212645 15988 nntpd lkg dec...
10,2,in articl <3mwf2b1w165w njcc wisdom bubbl org ...
